* C-shunted profiling
* Inbuilt-Symmetry model
* Uniform Initialization, Optimal Enclosure : 2dim=5x5 ; 4dim=3x3x3x3
* Parameter subspace : I : fixed : 800GHz
    - A : J1==J3 / J2 ; C1==C3 = 600GHz, C2 = 10GHz
    - B : C1==C3 / C2 ; J1==J3 = 150GHz, J2 = 100GHz
    - C : J1==J3 / C1==C3 ; J2 = 100GHz, C2 = 10GHz
    - D : J1==J3 / J2 / C1==C3 / C2
* Algorithms :
    - SGD : learning rate={.01,.0001} : SGD2,SGD4 :
    - Adam : learning rate={.01,.0001} : Adam2,Adam4
    - Nelder Mead : !!! Unconstrained Symmetry on Nelder Mead
    - Conjugate Gradient !!! Unconstrained Symmetry on Nelder Mead
    - L-BFGS :
        - scipy : *assummed Wolfe
        - torch : Wolfe
        - torch : lr = .0001
* Benchmarkers Optimization : multi-initialization
    - Loss : min,max,avg : Box Plots
    - Termination : count,steps:min,max,avg

* SGDx, Adam4 are too slow
* Wolfe torch LBFGS : stagnation

In [1]:
from multiprocess import Pool
#from multiprocessing import Pool
from DiSuQ.Torch.optimization import uniformParameters,initializationParallelism,truncNormalParameters,initializationSequential
from DiSuQ.Torch import models
from DiSuQ.Torch.components import J0,L0,C0
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossTransition
from scipy.optimize import LinearConstraint
from torch import tensor, float32 as float
from numpy import arange,linspace,array,argsort,vstack,isnan
from DiSuQ.utils import plotCompare,plotOptimization,plotBox
import pickle
from torch import set_num_threads
set_num_threads(32)

In [2]:
with open('../C-shuntedProfiling/target_fluxqubit.p', 'rb') as content: target_info = pickle.load(content)
target_spectrum = target_info['spectrum']
E10 = target_spectrum[:,1] - target_spectrum[:,0]
E20 = target_spectrum[:,2] - target_spectrum[:,0]
target = {'E10':E10[[0,20,-1]],'E20':E20[[0,20,-1]]}

In [3]:
Algo = ["Adam2","lrBFGS",'Nelder-Mead','LBFGS']

In [4]:
lossFunction = lossTransition(tensor(target['E10'],dtype=float),tensor(target['E20'],dtype=float))

In [5]:
iterations = 5; n_scape = 40

In [6]:
flux_range = linspace(0,1,3,endpoint=True)
flux_profile = [{'I':flux} for flux in flux_range]

In [7]:
symmetry = array([[1,0,0,0,-1,0,0],[0,1,0,0,0,-1,0]])
symmetry = LinearConstraint(symmetry,[0,0],[0,0])

In [8]:
def printLog(dLog):
    print(dLog['loss'].iloc[-1])

In [17]:
def benchmarking(optimizer,initials,subspace):
    Adam2,lrBFGS,NelMea,LBFGS = [],[],[],[]
    for index,parameter in enumerate(initials):
        print(index,'-------------------------------------------------')
        print(parameter)
        # scipy-BFGS
        print('L-BFGS')
        optimizer.circuit.initialization(parameter)  
        print(optimizer.circuitState())
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters(subspace)
        LBFGS.append(optimizer.minimization(lossFunction,flux_profile,subspace=subspace,
                                           method='L-BFGS-B',tol=1e-6,maxiter=iterations))
        
        # Adam
        print('Adam')
        optimizer.circuit.initialization(parameter)
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters(subspace)
        Adam2.append(optimizer.optimization(lossFunction,flux_profile,iterations=iterations,lr=.01))
        
#         optimizer.circuit.initialization(parameter)
#         optimizer.parameters,optimizer.IDs = optimizer.circuitParameters(subspace)
#         Adam4.append(optimizer.optimization(lossFunction,flux_profile,iterations=iterations,lr=.0001))
        
        # Nelder Mead
        print('Nelder Mead')
        optimizer.circuit.initialization(parameter)
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters(subspace)
        NelMea.append(optimizer.minimization(lossFunction,flux_profile,subspace=subspace,
                                             method='Nelder-Mead',tol=1e-6,maxiter=iterations))
        
#         Conjugate Gradient
#         print('Conjugate Gradient')
#         optimizer.circuit.initialization(parameter)
#         optimizer.parameters,optimizer.IDs = optimizer.circuitParameters(subspace)
#         CG.append(optimizer.minimization(lossFunction,flux_profile,subspace=subspace,
#         method='CG',tol=1e-8,maxiter=iterations))
        
#         # Wolfe BFGS
#         print('Wolfe BFGS')
#         optimizer.circuit.initialization(parameter)     
#         optimizer.parameters,optimizer.IDs = optimizer.circuitParameters(subspace)        
#         Wolfe.append(optimizer.optimizationLBFGS(lossFunction,flux_profile,iterations=iterations))
        
        # lr-LBFGS
        print('lr LBFGS')
        optimizer.circuit.initialization(parameter)
        optimizer.parameters,optimizer.IDs = optimizer.circuitParameters(subspace)
        lrBFGS.append(optimizer.optimizationLBFGS(lossFunction,flux_profile,iterations=iterations,lr=.1))
        
        
    return Adam2,lrBFGS,NelMea,LBFGS

In [10]:
def optimizationAnalysis(init,subspace,Search,success=1.):
    Loss,Success,Paths = [],[],[]
    for index,(init,(dLogs,dParams,dCircuit)) in enumerate(zip(init,Search)):
        Paths.append(dCircuit[subspace].to_numpy())
        loss = dLogs['loss'].to_numpy()
        Loss.append(loss[-1])
        if loss[-1] < success:
            Success.append(len(loss))
    return Paths,Loss,Success

In [11]:
def analysisPlotting(Optimization):
    paths = dict()
    losse = dict()
    for algo,(Paths,Loss,Success) in zip(Algo,Optimization):
        indices = argsort(Loss)[0]
        paths[algo] = Paths[indices]
        #for idx in indices:
        #    paths[algo+'-'+str(idx)] = Paths[idx]
        losse[algo] = Loss
    return paths,losse

In [12]:
def lossScapeBounds(paths):
    Paths = []
    for algo,path in paths.items():
        Paths.append(path)
    Paths = vstack(Paths)
    Paths = Paths[:, ~isnan(Paths).any(axis=0)]
    return Paths.min(0),Paths.max(0)

### simulation

#### subspace B

In [13]:
basis = {'O':[8],'J':[6,6],'I':[]}; rep = 'K'
C12 = .5 ; C22 = 20. ; C11 = C22
JJ1 = 150; JJ3 = 150 ; JJ2 = 50
ind = 800
circuit = models.shuntedQubit(basis,josephson=[JJ1,JJ2,JJ3],cap=[C11,C12,C22],ind=ind,sparse=False,symmetry=True)
static = circuit.circuitState()

In [14]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

In [15]:
subspace = ['C1','C2'] ; N = 5
initials = uniformParameters(circuit,subspace,N)
#initials = truncNormalParameters(circuit,subspace,N,var=5)
len(initials)

25

In [ ]:
OptimizationB = benchmarking(optimizer,initials,subspace)
Adam2,lrBFGS,NelMea,LBFGS = OptimizationB

0 -------------------------------------------------
{'JJ1': 150.00001525878906, 'C1': 416.6666666666667, 'JJ2': 50.0, 'C2': 416.6666666666667, 'JJ3': 150.00001525878906, 'C3': 20.0, 'I': 800.0}
L-BFGS
{'JJ1': 150.00003051757812, 'C1': 416.66668701171875, 'JJ2': 50.0, 'C2': 416.66668701171875, 'JJ3': 150.00003051757812, 'C3': 416.66668701171875, 'I': 800.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.10224D+06    |proj g|=  4.16667D+02

At iterate    1    f=  1.19770D+03    |proj g|=  2.08460D+01

At iterate    2    f=  2.60672D+02    |proj g|=  1.28481D+01

At iterate    3    f=  2.06467D+02    |proj g|=  3.57387D+01
  ys=-4.515E+00  -gs= 2.587E+00 BFGS update SKIPPED

At iterate    4    f=  1.01420D+02    |proj g|=  1.75742D+01

At iterate    5    f=  4.11713D+01    |proj g|=  5.06376D+01

At iterate    6    f=  3.07700D+01    |proj g|=  


 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.



At iterate   20    f=  2.24020D+00    |proj g|=  2.13437D+00

At iterate   21    f=  1.85299D+00    |proj g|=  3.09540D+00

At iterate   22    f=  2.12624D-01    |proj g|=  2.15326D+00
  ys=-1.518E+01  -gs= 9.226E+00 BFGS update SKIPPED

At iterate   23    f=  1.65854D-01    |proj g|=  7.27501D+00

At iterate   24    f=  9.55604D-02    |proj g|=  4.97807D-01

At iterate   25    f=  7.73374D-02    |proj g|=  4.26415D-01

At iterate   26    f=  7.54628D-02    |proj g|=  1.21477D+00

At iterate   27    f=  7.25947D-02    |proj g|=  4.51994D-02

At iterate   28    f=  7.25947D-02    |proj g|=  4.51994D-02



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2     28    115     31     4     0   4.520D-02   7.259D-02
  F =   7.2594702243804932E-002

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
Adam
Nelder Mead


...

/usr/local/lib/python3.11/site-packages/scipy/optimize/_minimize.py:549: RuntimeWarning:

Method Nelder-Mead does not use gradient information (jac).



Optimization terminated successfully.
         Current function value: 1261.081055
         Iterations: 36
         Function evaluations: 128
lr LBFGS
20 -------------------------------------------------
{'JJ1': 150.00001525878906, 'C1': 416.6666666666667, 'JJ2': 50.0, 'C2': 2083.3333333333335, 'JJ3': 150.00001525878906, 'C3': 20.0, 'I': 800.0}
L-BFGS
{'JJ1': 150.00003051757812, 'C1': 416.66668701171875, 'JJ2': 50.0, 'C2': 2083.333251953125, 'JJ3': 150.00003051757812, 'C3': 416.66668701171875, 'I': 800.0}
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.66107D+06    |proj g|=  2.08333D+03

At iterate    1    f=  1.19770D+03    |proj g|=  2.08460D+01

At iterate    2    f=  2.77375D+02    |proj g|=  7.11386D+00

At iterate    3    f=  2.47791D+02    |proj g|=  1.20398D+01
  ys=-2.249E-01  -gs= 3.714E-01 BFGS update SKIPPED

At iterate    4    f=


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


Nelder Mead


/usr/local/lib/python3.11/site-packages/scipy/optimize/_minimize.py:549: RuntimeWarning:

Method Nelder-Mead does not use gradient information (jac).



In [ ]:
Result = [optimizationAnalysis(initials,subspace,algo) for algo in OptimizationB]
paths,losse = analysisPlotting(Result)

In [ ]:
plotBox(losse,'Loss Distribution-C1-C2',export='pdf',size=(600,800))

In [ ]:
lower,upper = lossScapeBounds(paths)
C1 = linspace(lower[0],upper[0],n_scape,endpoint=True)
C2 = linspace(lower[1],upper[1],n_scape,endpoint=True)
scape = {'C1':C1,'C2':C2}

In [ ]:
flux_profile = [{'I':flux} for flux in flux_range]

In [ ]:
Loss = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [ ]:
plotOptimization(Loss,C1,C2,paths,'Optimization-C1_C2','C1(GHz)','C2(GHz)',export='pdf',size=(600,800),legend=False)

#### subspace C

In [ ]:
basis = {'O':[8],'J':[6,6],'I':[]}; rep = 'K'
C12 = .5 ; C22 = 20. ; C11 = C22
JJ1 = 150; JJ3 = 150 ; JJ2 = 50
ind = 800
circuit = models.shuntedQubit(basis,josephson=[JJ1,JJ2,JJ3],cap=[C11,C12,C22],ind=ind,sparse=False,symmetry=True)
static = circuit.circuitState()

In [ ]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

In [ ]:
subspace = ['JJ1','C1'] ; N = 5
initials = uniformParameters(circuit,subspace,N)
#initials = truncNormalParameters(circuit,subspace,N,var=5)
len(initials)

In [ ]:
OptimizationC = benchmarking(optimizer,initials,subspace)
Adam2,lrBFGS,NelMea,LBFGS = OptimizationC

In [ ]:
Result = [optimizationAnalysis(initials,subspace,algo) for algo in OptimizationC]
paths,losse = analysisPlotting(Result)

In [ ]:
plotBox(losse,'Loss Distribution-JJ1-C1',export='pdf',size=(600,800))

In [ ]:
lower,upper = lossScapeBounds(paths)
JJ1 = linspace(lower[0],upper[0],n_scape,endpoint=True)
C1 = linspace(lower[1],upper[1],n_scape,endpoint=True)
scape = {'JJ1':JJ1,'C1':C1}

In [ ]:
flux_profile = [{'I':flux} for flux in flux_range]

In [ ]:
Loss = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [ ]:
plotOptimization(Loss,JJ1,C1,paths,'Optimization-JJ1_C1','JJ1(GHz)','C1(GHz)',export='pdf',size=(600,800),legend=False)

#### subspace D

In [ ]:
basis = {'O':[8],'J':[6,6],'I':[]}; rep = 'K'
C12 = .5 ; C22 = 20. ; C11 = C22
JJ1 = 150; JJ3 = 150 ; JJ2 = 50
ind = 800
circuit = models.shuntedQubit(basis,josephson=[JJ1,JJ2,JJ3],cap=[C11,C12,C22],ind=ind,sparse=False,symmetry=True)
static = circuit.circuitState()

In [ ]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

In [ ]:
subspace = ['JJ1','JJ2','C1','C2'] ; N = 2
initials = uniformParameters(circuit,subspace,N)
#initials = truncNormalParameters(circuit,subspace,N,var=5)
len(initials)

In [ ]:
OptimizationD = benchmarking(optimizer,initials,subspace)
Adam2,lrBFGS,NelMea,LBFGS = OptimizationD

In [ ]:
Result = [optimizationAnalysis(initials,subspace,algo) for algo in OptimizationD]
paths,losse = analysisPlotting(Result)

In [ ]:
plotBox(losse,'Loss Distribution',export='pdf',size=(600,800))

#### subspace A

In [ ]:
basis = {'O':[8],'J':[6,6],'I':[]}; rep = 'K'
C12 = .5 ; C22 = 20. ; C11 = C22
JJ1 = 150; JJ3 = 150 ; JJ2 = 50
ind = 800
circuit = models.shuntedQubit(basis,josephson=[JJ1,JJ2,JJ3],cap=[C11,C12,C22],ind=ind,sparse=False,symmetry=True)
static = circuit.circuitState()

In [ ]:
optimizer = optimization.OrderingOptimization(circuit,representation=rep)

In [ ]:
subspace = ['JJ1','JJ2'] ; N = 5
initials = uniformParameters(circuit,subspace,N)
#initials = truncNormalParameters(circuit,subspace,N,var=5)
len(initials)

In [ ]:
OptimizationA = benchmarking(optimizer,initials,subspace)
Adam2,lrBFGS,NelMea,LBFGS = OptimizationA

In [ ]:
Result = [optimizationAnalysis(initials,subspace,algo) for algo in OptimizationA]
paths,losse = analysisPlotting(Result)

In [ ]:
plotBox(losse,'Loss Distribution-JJ1-JJ2',export='pdf',size=(600,800))

In [ ]:
lower,upper = lossScapeBounds(paths)
JJ1 = linspace(lower[0],upper[0],n_scape,endpoint=True)
JJ2 = linspace(lower[1],upper[1],n_scape,endpoint=True)
scape = {'JJ1':JJ1,'JJ2':JJ2}

In [ ]:
flux_profile = [{'I':flux} for flux in flux_range]

In [ ]:
Loss = optimizer.lossScape(lossFunction,flux_profile,scape,static)

In [ ]:
plotOptimization(Loss,JJ1,JJ2,paths,'Optimization-JJ1_JJ2','JJ1(GHz)','JJ2(GHz)',export='pdf',size=(600,800),legend=False)